## Used Vehicle Inventory - Project <a id='intro'></a>

## Introduction <a id='intro'></a>

In this project, the main aim is to develop and produce a web application to a cloud service (Render) and make it accessible to the public. The dataset provided contains information on car sales advertisements from 1908 to 2019.

### Stages 
Data on car sale advertisements is stored in a csv file titled '../vehicles_us.csv'. There is no information about the quality of the data, so data exploration is needed it before doing the analysis.

First, evaluate the quality of the data and see whether its issues are significant. Then, during data preprocessing, try to account for the most critical problems. Finally plot some charts to display patterns and trends found.

In [1]:
import pandas as pd
import plotly.express as px
import streamlit as st


In [2]:
vehicles_df = pd.read_csv('../vehicles_us.csv')

In [3]:
vehicles_df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [4]:
(vehicles_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Judging from the output of the .info() method, there are a few columns within the dataset the have missing values. With data integrity and analysis in mind, I think it would be best to delete those records. I think changing the 'paint_color' column null values maybe suitable given that it's a categorical variable. For reasons that could alter the overall portrayal of the value of a vehicle, filling in the null values with a placeholder can mess with the results of the analysis. Also, the 'date_posted' and 'model year' column can be converted to datetime data types. Finally, I think it's worth investigating the 'is_4wd' column to better understand the values within that column.

#### Check for Duplicates

In [5]:
vehicles_df.duplicated().sum()  # Count of duplicate rows


0

Based on the results of the duplicated method, it seems that this dataset doesn't hold any duplicates.

In [6]:
print(vehicles_df.isna().sum())
print(vehicles_df.info())

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null

There seems to be missing values in certain columns of the dataframe.

#### Understanding "is_4wd" Column

In [7]:
vehicles_df['is_4wd'].unique()

array([ 1., nan])

My intuition leads to believe  that this column was intended to let viewers know that the vehicle is a 4 wheel drive or not. With that being said, I think it would be best to convert this column to a string and change all of the '1.0' values to 'yes' and nans to 'no' (given that this is binary question).

In [8]:
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].astype('str')
vehicles_df['is_4wd'].info() # a check to see if the data type conversion was successful
vehicles_df.sample(5)
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].replace('1.0', 'yes') # Replaces old values with new values
vehicles_df['is_4wd'] = vehicles_df['is_4wd'].replace('nan', 'no')
vehicles_df.head(5)

<class 'pandas.core.series.Series'>
RangeIndex: 51525 entries, 0 to 51524
Series name: is_4wd
Non-Null Count  Dtype 
--------------  ----- 
51525 non-null  object
dtypes: object(1)
memory usage: 402.7+ KB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,yes,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,yes,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,no,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,no,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,no,2019-04-02,28


#### Changing Null Values in 'paint_color' column

In [9]:
vehicles_df['paint_color'] = vehicles_df['paint_color'].fillna(value='text')  # Fill missing values
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  object 
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(3), int64(2), object(8)
memory usage: 5.1+ MB


#### Drop Null Values

In [10]:
#vehicles_df.info()
vehicles_df = vehicles_df.dropna()  # Drop rows with missing values
vehicles_df
vehicles_df = vehicles_df.reset_index()
vehicles_df = vehicles_df.drop(columns=['index'])  # Drop specific columns
vehicles_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36419 entries, 0 to 36418
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         36419 non-null  int64  
 1   model_year    36419 non-null  float64
 2   model         36419 non-null  object 
 3   condition     36419 non-null  object 
 4   cylinders     36419 non-null  float64
 5   fuel          36419 non-null  object 
 6   odometer      36419 non-null  float64
 7   transmission  36419 non-null  object 
 8   type          36419 non-null  object 
 9   paint_color   36419 non-null  object 
 10  is_4wd        36419 non-null  object 
 11  date_posted   36419 non-null  object 
 12  days_listed   36419 non-null  int64  
dtypes: float64(3), int64(2), object(8)
memory usage: 3.6+ MB


#### Date Type Conversion

In [11]:
vehicles_df['date_posted'] = pd.to_datetime(vehicles_df['date_posted'], format='%Y-%m-%d')
vehicles_df['model_year'] = pd.to_datetime(vehicles_df['model_year'], format='%Y')
vehicles_df['model_year'] = vehicles_df['model_year'].dt.to_period('Y')
vehicles_df.info()
vehicles_df.head()

   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36419 entries, 0 to 36418
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         36419 non-null  int64         
 1   model_year    36419 non-null  period[Y-DEC] 
 2   model         36419 non-null  object        
 3   condition     36419 non-null  object        
 4   cylinders     36419 non-null  float64       
 5   fuel          36419 non-null  object        
 6   odometer      36419 non-null  float64       
 7   transmission  36419 non-null  object        
 8   type          36419 non-null  object        
 9   paint_color   36419 non-null  object        
 10  is_4wd        36419 non-null  object        
 11  date_posted   36419 non-null  datetime64[ns]
 12  days_listed   36419 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(7), period[Y-DEC](1)
memory usage: 3.6+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6.0,gas,145000.0,automatic,SUV,text,yes,2018-06-23,19
1,5500,2013,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,no,2019-02-07,79
2,14900,2017,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,no,2019-04-02,28
3,14990,2014,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,yes,2018-06-20,15
4,12990,2015,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,no,2018-12-27,73


In [12]:
vehicles_df = vehicles_df.loc[vehicles_df['fuel'] != 'other']  # Filter rows based on condition
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36349 entries, 0 to 36418
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         36349 non-null  int64         
 1   model_year    36349 non-null  period[Y-DEC] 
 2   model         36349 non-null  object        
 3   condition     36349 non-null  object        
 4   cylinders     36349 non-null  float64       
 5   fuel          36349 non-null  object        
 6   odometer      36349 non-null  float64       
 7   transmission  36349 non-null  object        
 8   type          36349 non-null  object        
 9   paint_color   36349 non-null  object        
 10  is_4wd        36349 non-null  object        
 11  date_posted   36349 non-null  datetime64[ns]
 12  days_listed   36349 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(7), period[Y-DEC](1)
memory usage: 3.9+ MB


After seeing the various types of fuel, I don't think it would be beneficial to include cars that have fuel sources labeled as 'other'. Making an accurate deduction for each record labeled with the 'other' is nearly impossible because a guess has to be made out of 4 other options. 

In [13]:
avg_price_per_year = vehicles_df.groupby('model_year')['price'].mean()
avg_price_per_year = avg_price_per_year.reset_index()
avg_price_per_year
avg_price_per_year['model_year'] = avg_price_per_year['model_year'].astype(str)
avg_price_per_year
   


,model_year,price
0,1908,12995.000000
1,1936,5000.000000
2,1948,21000.000000
3,1955,23900.000000
4,1958,36450.000000
...,...,...
60,2015,18383.228422
61,2016,19574.220549
62,2017,20923.394289
63,2018,24637.362618


In [14]:
px.line(avg_price_per_year, x='model_year', y='price', title='AVG Price per Model Year')




The graph shows that the model year isn't the only variable that is considered when generating the price of a used vehicle.

In [15]:
px.scatter(avg_price_per_year, x='model_year', y='price', title='AVG Price per Model Year')

In [16]:
vehicles_condition = vehicles_df.groupby('condition')['price'].mean()
vehicles_condition = vehicles_condition.reset_index()
print(vehicles_condition)
px.bar(vehicles_condition, x='condition', y='price', title='AVG Price of a Vehicle per Condition Class',color='condition', color_discrete_sequence=['red', 'blue', 'green', 'yellow', 'purple', 'orange'])



   condition         price
0  excellent  12856.784973
1       fair   3294.109852
2       good  10925.068129
3   like new  16684.189342
4        new  27792.133333
5    salvage   4226.862500


Based on the dataset, it seems that the average price for 'new' cars is the highest. This may be due to the fact that this dataset may have a higher share of 'new' cars thant any other car condition. Lets see.

In [17]:
vehicle_condition_count = vehicles_df['condition'].value_counts()
vehicle_condition_sum = len(vehicles_df['condition'])
v_c_1 = vehicle_condition_count['new']
new_share = v_c_1 / vehicle_condition_sum
print('Share of new:', "%.3f" % new_share)

v_c_2 = vehicle_condition_count['excellent']
excel_share = v_c_2 / vehicle_condition_sum
print('Share of excellent:', "%.3f" % excel_share)

Share of new: 0.003
Share of excellent: 0.480


After doing some investigation, it seems that the share of 'new' cars doesn't seem to be the reason why the average price for the 'new' category is the highest.

In [18]:
fig = px.histogram(vehicles_df, x='odometer')
fig.update_layout(width=800, height=400, margin=dict(l=50, r=50, t=50, b=50))
fig


It seems that this dealership receive cars that fall between 98k and 121k more frequent than other cars with other odometer readings. Along with majority of the cars within the inventory having odometer readings above 121k.

In [19]:
vehicles_4wd = vehicles_df.groupby('is_4wd')['price'].mean()
vehicles_4wd = vehicles_4wd.reset_index()
vehicles_4wd
px.bar(vehicles_4wd, x='is_4wd', y='price', title='Does 4 WD increase the AVG Price of a Used Vehicle', color='is_4wd', color_discrete_sequence=['cyan', 'brown'])

On average, it seems that cars that have 4 wheel drives are priced higher.

In [20]:
vehicles_fuel = vehicles_df.groupby('fuel')['price'].mean()
vehicles_fuel = vehicles_fuel.reset_index()
print(vehicles_df['fuel'].value_counts())
print(vehicles_fuel)
px.bar(vehicles_fuel, x='fuel', y='price', title='What is AVG Price of a Used Vehicle based on various fuel type?', color='fuel', color_discrete_sequence=['red', 'blue', 'green', 'yellow']) 

fuel
gas         33475
diesel       2586
hybrid        285
electric        3
Name: count, dtype: int64
       fuel         price
0    diesel  22362.911060
1  electric   4967.000000
2       gas  11430.352054
3    hybrid   7528.859649


Vehicles that require diesel fuel hold significantly more value than vehicles that require othe fuel types.

In [21]:
px.bar(vehicles_df, x='days_listed', y='price',width=500) # This graph is similar to px.histogram(vehicles_df, x='days_listed')

Judging from the histogram, majority of the vehicles have been on the lot more than 22 days and the prime time to sell a car is around 16 to 22 days.

## Conclusion <a id='hypotheses'></a>

Based on our analysis, the vehicles that seem the most profitable for the used vehicle dealership are the ones that run on desiel, been on the lot between 16 to 22 days, have 4 wheel drive or have an odometer reading between 98k and 121k.